### Collabrator: Steven Kim

# Question 1

(20 points) Design and implement an iterative Power Method approach to
determine the first principal component of the PCA transformation.

# Question 2

(20 points) Design and implement an iterative Power Method
approach to determine the second principal component of the PCA
transformation.

# Problem Number 3

 (10 points) Describe the benefit of a gradient approach w.r.t.
the SVD approach for the PCA transformation

#### There are some reasons:

1. PCA wants to do the eigen-decomposition of X.T * X to get the d's biggest eigen vectors, and SVD can calculate these. In scikit-learn, SVD is a part in PCA process.
2. When the dataset is large, it is computing-comsuming to calculate the covariance matrix in PCA. Using SVD would be much faster than simply calculate the eigrn-decomposition of X.T * X.
3. SVD is more accurate than eigenvalue Decomposition of covariance matrix.

# Question 4

(20 points) Using a binary classifier (logistic regression or
SVM) please implement in python both of the 2 different approaches
(one vs. one, one vs. all) to handle the problem of more
than two different classes (use the iris dataset). For
reference use the following link
https://en.wikipedia.org/wiki/Multiclass_classification


In [9]:
import pandas as pd
irisdata=pd.read_csv('iris.csv',header=0)
irisdata.columns=['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
#irisdata=irisdata.drop(columns=['class'])

irisdata.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
# one vs one
from sklearn.preprocessing import LabelBinarizer
y_dense = LabelBinarizer().fit_transform(irisdata['class'])

# Question 5

(20 points) Extensively describe the overfitting and
underfitting problem. Use execution examples with a decision
tree and SVM (with or without kernel). Use the scikit
implementations. Show underfitting, good behavior, and overfitting examples.

Solution: 

**Overfitting** occurs when a model is too closely aligned to a limited set of data points. As a result, the model is useful in reference only to its initial data set, and not applicable to any other data sets.

**Underfitting** occurs when a model is unable to learn much from the training data. Therefore, the results of the application of the model in the training data or any other data sets are both not good.

Because iris dataset is too small, I use the titanic data set to show the examples of overfitting, good behavior and underfitting.

In [112]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
df_train = pd.read_csv('train.csv')
df_comb = df_train
X = pd.DataFrame()

def encode_sex(x):
    return 1 if x == 'female' else 0

def family_size(x):
    size = x.SibSp + x.Parch 
    return 4 if size > 3 else size

X['Sex'] = df_comb.Sex.map(encode_sex)
X['Pclass'] = df_comb.Pclass
X['FamilySize'] = df_comb.apply(family_size, axis=1)
fare_median = df_train.groupby(['Sex', 'Pclass']).Fare.median()
fare_median.name = 'FareMedian'

age_mean = df_train.groupby(['Sex', 'Pclass']).Age.mean()
age_mean.name = 'AgeMean'

def join(df, stat):
    return pd.merge(df, stat.to_frame(), left_on=['Sex', 'Pclass'], right_index=True, how='left')

X['Fare'] = df_comb.Fare.fillna(join(df_comb, fare_median).FareMedian)
X['Age'] = df_comb.Age.fillna(join(df_comb, age_mean).AgeMean)
def quantiles(series, num):
    return pd.qcut(series, num, retbins=True)[1]

def discretize(series, bins):
    return pd.cut(series, bins, labels=range(len(bins)-1), include_lowest=True)
    
X['Fare'] = discretize(X.Fare, quantiles(df_comb.Fare, 10))
X['Age'] = discretize(X.Age, quantiles(df_comb.Age, 10))

,Sex,Pclass,FamilySize,Fare,Age
0,0,3,1,0,2
1,1,1,1,8,7
2,1,3,0,2,4
3,1,1,1,8,6
4,0,3,0,2,6
...,...,...,...,...,...
886,0,2,0,4,4
887,1,1,0,7,1
888,1,3,3,6,2
889,0,1,0,7,4


## SVM

In [154]:
y=df_train.Survived

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=424)

from sklearn import svm
from sklearn.metrics import mean_squared_error,accuracy_score,precision_score,classification_report,confusion_matrix


clf=svm.SVC( C=0.00000001)
clf.fit(X_train,y_train)
y_hat=clf.predict(X_test)
X_hat=clf.predict(X_train)
print('The underfitting situation:')
print('The accuracy in Training data set is:',accuracy_score(y_train,X_hat))
print('The accuracy in Testing data set is:',accuracy_score(y_test,y_hat))
print('The result show that the accuracy in both Training data and Testing data are both low, shows the model is underfitting')


clf=svm.SVC( C=1000)
clf.fit(X_train,y_train)
y_hat=clf.predict(X_test)
X_hat=clf.predict(X_train)
print('The overfitting situation:')
print('The accuracy in Training data set is:',accuracy_score(y_train,X_hat))
print('The accuracy in Testing data set is:',accuracy_score(y_test,y_hat))
print('The result show that the accuracy in Training data is much higher than in test data, that means the model is overfitting')


clf=svm.SVC( C=1)
clf.fit(X_train,y_train)
y_hat=clf.predict(X_test)
X_hat=clf.predict(X_train)
print('The good behavior situation:')
print('The accuracy in Training data set is:',accuracy_score(y_train,X_hat))
print('The accuracy in Testing data set is:',accuracy_score(y_test,y_hat))
print('The result show that the accuracy in Training data is similar to test data, that means the model is in good behavior')

The underfitting situation:
The accuracy in Training data set is: 0.6067415730337079
The accuracy in Testing data set is: 0.6302521008403361
The result show that the accuracy in both Training data and Testing data are both low, shows the model is underfitting
The overfitting situation:
The accuracy in Training data set is: 0.8857677902621723
The accuracy in Testing data set is: 0.8179271708683473
The result show that the accuracy in Training data is much higher than in test data, that means the model is overfitting
The good behavior situation:
The accuracy in Training data set is: 0.8295880149812734
The accuracy in Testing data set is: 0.7815126050420168
The result show that the accuracy in Training data is similar to test data, that means the model is in good behavior


## Decision Tree

In [172]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(random_state=0,max_depth=100,max_leaf_nodes=100)
clf1.fit(X_train,y_train)
y_hat=clf1.predict(X_test)
X_hat=clf1.predict(X_train)
print('The overfitting situation:')
print('The accuracy in Training data set is:',accuracy_score(y_train,X_hat))
print('The accuracy in Testing data set is:',accuracy_score(y_test,y_hat))
print('The result show that the accuracy in Training data is much higher than in test data, that means the model is overfitting')


clf1 = DecisionTreeClassifier(random_state=0,max_depth=1,max_leaf_nodes=3)
clf1.fit(X_train,y_train)
y_hat=clf1.predict(X_test)
X_hat=clf1.predict(X_train)
print('The underfitting situation:')
print('The accuracy in Training data set is:',accuracy_score(y_train,X_hat))
print('The accuracy in Testing data set is:',accuracy_score(y_test,y_hat))
print('The result show that the accuracy in both Training data and Testing data are both low, shows the model is underfitting')



The overfitting situation:
The accuracy in Training data set is: 0.9176029962546817
The accuracy in Testing data set is: 0.7843137254901961
The result show that the accuracy in Training data is much higher than in test data, that means the model is overfitting
The underfitting situation:
The accuracy in Training data set is: 0.7921348314606742
The accuracy in Testing data set is: 0.7787114845938375
The result show that the accuracy in both Training data and Testing data are both low, shows the model is underfitting


# Question 6
(10 points) Show examples when the use of
kernel procedure is more efficient in terms of training and
prediction computational time w.r.t. polynomial features
transformation

In [59]:
data = pd.read_csv('iris.csv',sep=',').values
X=data[:,0:4]
y=data[:,4]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=424)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

from sklearn import svm
import time

start = time.time()
clf1=svm.SVC(kernel='poly')
clf1.fit(X_train,y_train)
clf1.predict(X_test)
end = time.time()
print('Time needed for Poly Kernel',end-start)

start = time.time()
clf1=svm.SVC(kernel='rbf')
clf1.fit(X_train,y_train)
clf1.predict(X_test)
end = time.time()
print(end-start)

start = time.time()
clf1=svm.SVC(kernel='linear')
clf1.fit(X_train,y_train)
clf1.predict(X_test)
end = time.time()
print(end-start)

start = time.time()
clf1=svm.SVC()
clf1.fit(X_train,y_train)
clf1.predict(X_test)
end = time.time()
print(end-start)

 0.001415252685546875
0.002007007598876953
0.001425027847290039
0.001538991928100586


# Question 7

(10 points) Write a procedure to estimate in the SVC
classifier (in Scikit) the best kernel (RBF, Polynomial,
sigmoid), the best gamma & degree, and the best C. Use the
grid search without implement it. Use the following
reference:

    a. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
    
    b. https://scikit-learn.org/stable/modules/grid_search.html

In [72]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':( 'rbf','poly','sigmoid'), 
              'C':[1,2,3,4,5,6,7,8,9,10],
              'gamma':('scale', 'auto'),
              'degree':[1,2,3,4,5,6,7,8,9]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)


GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'gamma': ('scale', 'auto'),
                         'kernel': ('rbf', 'poly', 'sigmoid')})

In [73]:
#sorted(clf.cv_results_.keys())
clf.best_params_
#help(svm.SVC())

{'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}

 # Question 8
 (20 points) Create examples to explain the property and the
importance of the following kernels:
    
    a. https://en.wikipedia.org/wiki/Graph_kernel
    
    b. https://en.wikipedia.org/wiki/String_kernel

    c. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.sigmoid_kernel.html

Graph Kernel